Here we will do a  simple prediction of housing regression data run using pandas , sklearn and XGBoost .
I wil present the code implementation and not a detailed charting analysis as others have done lot of data EDA for this dataset and its ot a very complex dataset to begin with and very small also 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
## i like declaring imports when needed at the place 
import pandas as pd
import numpy as np


In [3]:
df_train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
df_test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
df_test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


Extract the Target variable from Train set ,and remove irrelevant columns


In [5]:
## first step we remove the target variable from train data , and being both train and test df n line

df_y = df_train.SalePrice ## this willbecome the y training var later

df_train.drop('SalePrice',axis=1,inplace=True)

## we drop the id columns and MoSold and  mo year columns , reason they are sold dates not buy dates 
## they mght have an impact but our train does not know when the house will be sold 

df_train.drop(['Id','MoSold','YrSold'],axis=1,inplace=True)
df_test.drop(['Id','MoSold','YrSold'],axis=1,inplace=True)

print(df_train.shape,df_test.shape,df_y.shape)


(1460, 77) (1459, 77) (1460,)


Data Cleaning and Corrections 

In [6]:
## start with aalysisng and cleaning training andtest data for normal irregularities
## get a list of numerical var and categorical var will come in handy 
c_var = df_train.columns[df_train.dtypes=='object']  ## max data rows = 1460
n_var = df_train.columns[df_train.dtypes != 'object']  ## max data rows = 1460
print(c_var.values)
print(n_var.values) 



['MSZoning' 'Street' 'Alley' 'LotShape' 'LandContour' 'Utilities'
 'LotConfig' 'LandSlope' 'Neighborhood' 'Condition1' 'Condition2'
 'BldgType' 'HouseStyle' 'RoofStyle' 'RoofMatl' 'Exterior1st'
 'Exterior2nd' 'MasVnrType' 'ExterQual' 'ExterCond' 'Foundation'
 'BsmtQual' 'BsmtCond' 'BsmtExposure' 'BsmtFinType1' 'BsmtFinType2'
 'Heating' 'HeatingQC' 'CentralAir' 'Electrical' 'KitchenQual'
 'Functional' 'FireplaceQu' 'GarageType' 'GarageFinish' 'GarageQual'
 'GarageCond' 'PavedDrive' 'PoolQC' 'Fence' 'MiscFeature' 'SaleType'
 'SaleCondition']
['MSSubClass' 'LotFrontage' 'LotArea' 'OverallQual' 'OverallCond'
 'YearBuilt' 'YearRemodAdd' 'MasVnrArea' 'BsmtFinSF1' 'BsmtFinSF2'
 'BsmtUnfSF' 'TotalBsmtSF' '1stFlrSF' '2ndFlrSF' 'LowQualFinSF'
 'GrLivArea' 'BsmtFullBath' 'BsmtHalfBath' 'FullBath' 'HalfBath'
 'BedroomAbvGr' 'KitchenAbvGr' 'TotRmsAbvGrd' 'Fireplaces' 'GarageYrBlt'
 'GarageCars' 'GarageArea' 'WoodDeckSF' 'OpenPorchSF' 'EnclosedPorch'
 '3SsnPorch' 'ScreenPorch' 'PoolArea' 'MiscVal']


In [7]:
## lots of na values in data 
print(df_train.isna().sum().sum(),  df_test.isna().sum().sum())


6965 7000


In [8]:
## we now prepare to impute numerical values using code to locate numerica data with na values
## we can use the simpleimputer class from sklearn however here we will do it manually in pandas for 
## simplicity purpose. for numerical valiables first we will calculate the mean values 
## and replace them for mssing na or null values 


print("before train process :" , df_train[n_var].isna().sum().sum() )

## for train data 
for idx,val in enumerate(n_var):
  if df_train[val].isna().sum() > 0 :
    ser = df_train[val].fillna(df_train[val].mean()) ## create series for na mean values
    df_train.drop(val,axis=1,inplace=True)
    df_train[val] = ser 
    
print("after train process :" , df_train[n_var].isna().sum().sum() )


## for test data 

print("before test process :" , df_test[val].isna().sum().sum() )

for idx,val in enumerate(n_var):
  if df_test[val].isna().sum() > 0 :
    ser = df_test[val].fillna(df_test[val].mean()) ## create series for na mean values
    df_test.drop(val,axis=1,inplace=True)
    df_test[val] = ser 
    
print("after test process :" , df_test[n_var].isna().sum().sum() )


before train process : 348
after train process : 0
before test process : 0
after test process : 0


Our Numerical data is cleaned up and imputed now

In [9]:
## now we know there are no null or NA values for numerical factors 1460 not null
print(df_train[n_var].isna().sum().sum(),df_test[n_var].isna().sum().sum() )
print(df_train.shape,df_test.shape)

0 0
(1460, 77) (1459, 77)


NOw we start with Categorical data for cleanup which is more time consuming

In [10]:
## check for na values in categorical var
print(df_train[c_var].isna().sum().sum())  ## gives us the null values for categoricall data , we will impute these
print(df_test[c_var].isna().sum().sum())  ## gives us the null values for categoricall data , we will impute these


6617
6670


In [11]:
## there are some columns with large NaN data more than 30-40% this is unusual and while we can impute it, 
## a better and IRL approcah is to remove this kind of data to lower model descrepency  or 
## fill the data with max co-occurance features  . here we will do the same and 
## replace categorical data Na values with max occurance for the feature

print("----------------train data ---------------------")

print("before train process", df_train[c_var].isna().sum().sum())

## lets find out na values in data 
for idx,val in enumerate(c_var):
  if df_train[val].isna().sum()>0:
    ## there are null values in columns
    most_val = df_train[val].value_counts(dropna=True).index[0] ## take the most occuring non na element from columns 
    ser = df_train[val].fillna(most_val) ## create series for na mean values
    df_train.drop(val,axis=1,inplace=True)
    df_train[val] = ser 

print("after train process :", df_train[c_var].isna().sum().sum())


## now for test data same processing 
print("----------------test data ---------------------")

print("before train process",df_test[c_var].isna().sum().sum())

## lets find out na values in data 
for idx,val in enumerate(c_var):
  if df_test[val].isna().sum()>0:
    ## there are null values in columns
    most_val = df_test[val].value_counts(dropna=True).index[0] ## take the most occuring non na element from columns 
    ser = df_test[val].fillna(most_val) ## create series for na mean values
    df_test.drop(val,axis=1,inplace=True)
    df_test[val] = ser 

print("after test process :", df_test[c_var].isna().sum().sum())

print(df_train.shape,df_test.shape)

----------------train data ---------------------
before train process 6617
after train process : 0
----------------test data ---------------------
before train process 6670
after test process : 0
(1460, 77) (1459, 77)


In [12]:
## finally we can see our datset in numerical and working is all cleaned and free of na values
## as a final check we will see unique values in category copy to check if there is any NA values missed out
## we can do it simply by comparing uniques

for idx,val in enumerate(df_train.columns):
  if df_train[val].nunique() != df_train[val].nunique(dropna=False):
    print("val:",val,"|",df_train[val].nunique(),"|",df_train[val].nunique(dropna=False))

for idx,val in enumerate(df_test.columns):
  if df_test[val].nunique() != df_test[val].nunique(dropna=False):
    print("val:",val,"|",df_test[val].nunique(),"|",df_test[val].nunique(dropna=False))

## no records we have clean data

In [13]:
## as category convertion before we convert all data to same case UPPER here to remove any typo errors
## this is not needed, but is a good practice if dealing with words , cause people type wrong 

## for train data 
for idx,val in enumerate(c_var):  
    ser = df_train[val].astype(str).str.upper()  ## create series for na mean values
    df_train.drop(val,axis=1,inplace=True)
    df_train[val] = ser 

print("after process :" , df_train[val].isna().sum().sum() )

## for test data 
for idx,val in enumerate(c_var):
    ser = df_test[val].astype(str).str.upper()  ## create series for na mean values
    df_test.drop(val,axis=1,inplace=True)
    df_test[val] = ser 

print("after process :" , df_test[val].isna().sum().sum() )

  
    

after process : 0
after process : 0


## Data Transformation

Nowwe apply data transformationa and encoding before final feeding to network 

In [14]:
## There are various ways to encode data , converting to 1-hot encoding or Lael, or Ordinal
## we will convert categorical features  with sklearn ordinal encoding to simply because it keeps our feature count fied to variable
## and saves lot of time and space in processing


from sklearn.preprocessing import OrdinalEncoder

##
df_train_cat = df_train[c_var]
df_test_cat = df_test[c_var]

ord = OrdinalEncoder()

train_ord = ord.fit_transform(df_train_cat) ## remember fit-transform on train, and only transform on test, most ppl make mistake
test_ord = ord.transform(df_test_cat)

print(train_ord.shape,test_ord.shape)

## the shapes are correct and in sync




(1460, 43) (1459, 43)


In [15]:
## now that the category features are encoded we will transform a second array with numerical features and scalng them
from sklearn.preprocessing import StandardScaler

df_train_num = df_train[n_var]
df_test_num = df_test[n_var]

print(df_train_num.shape,df_test_num.shape)
scalar = StandardScaler()

train_num = scalar.fit_transform(df_train_num)
test_num = scalar.transform(df_test_num)

print(train_num.shape,test_num.shape)




(1460, 34) (1459, 34)
(1460, 34) (1459, 34)


In [16]:
## now we have our data encoded into 2 nice seperate numpy arrays, we wil now combine them for processing and the final dataset

x_train = np.concatenate((train_num,train_ord),axis=1)
x_test = np.concatenate((test_num,test_ord),axis=1)

print(x_train.shape,x_test.shape)
## data looks good to go at this point 



(1460, 77) (1459, 77)


In [17]:
## train test split and we are good to go 
from sklearn.model_selection import train_test_split
## remember the y col that we split in beginnng of sheet y-trin is there
y_train = df_y.to_numpy()

x_train,x_val,y_train,y_val = train_test_split(x_train,y_train,test_size=0.2,train_size=0.8)
print(x_train.shape,x_val.shape,y_train.shape,y_val.shape)


(1168, 77) (292, 77) (1168,) (292,)


## Model Building

In [18]:
## now for model we will use XGBoost , cause its the best there is apart from NN models
## for XGB we still need some processing one, data and parameter tuning

from sklearn.model_selection import GridSearchCV 
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error,r2_score


### MOdel Parameter Tuning
GridSerachCV is he preferred method for tuning model parameters we wil use same 

In [19]:
## we will first use the grid serach CV for determining optimum hyperparameters using 2 params to optimize 
## can use more, just deends on the time taken 
params = {
    'learning_rate': np.arange(0.01, 0.10, 0.01)
    #,'n_estimators' : [500,1000]
    #, 'max_depth' : [5,10]    
    }

params

{'learning_rate': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09])}

In [20]:
## define the parametr optimizer
optimizer = GridSearchCV(
    XGBRegressor(
      n_estimators=500, max_depth=4, 
      min_child_weight=3, gamma=0, colsample_bytree=0.4, 
      subsample=0.7, learning_rate=0.05, verbosity=0, objective ='reg:squarederror'
      ), 
    params,cv=5
    )


In [21]:

optimizer.fit(x_train, y_train)

GridSearchCV(cv=5,
             estimator=XGBRegressor(colsample_bytree=0.4, learning_rate=0.05,
                                    max_depth=4, min_child_weight=3,
                                    n_estimators=500,
                                    objective='reg:squarederror', subsample=0.7,
                                    verbosity=0),
             param_grid={'learning_rate': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09])})

In [22]:
## get best params from grid serach

print(optimizer.best_params_)
## save the param run sin a DF
opt_params = pd.DataFrame(optimizer.cv_results_)

{'learning_rate': 0.04}


In [23]:
## lets get the validation scores on the data , we an also use the inbuilt eval_metric in XGB boost for this

y_pred = optimizer.predict(x_val) 
msescore = mean_squared_error(y_val, y_pred)
r2score = r2_score(y_val,y_pred)
print("mse=",msescore,"--r2 = ",r2score)
## looks good 

mse= 2027867262.88084 --r2 =  0.7133887840060089


## Model Prediction Submission

Now last part use the best params and submit the model  

In [24]:
## define the XGB regressor model parameters

## now we train model

model_params = {
    'colsample_bytree': 0.4,
    'n_estimators': 500 ,
    'min_child_weight': 3,
    'max_depth': 5 ,
    'subsample': 0.4,
    'learning_rate': optimizer.best_params_['learning_rate'],
    'gamma': 0,
    'reg_lambda': 0.02,
    'verbosity' : 2,
    'objective' : 'reg:squarederror'
}
model = XGBRegressor(tree_method='hist', **model_params)


In [25]:
## fit the model ## remember ghere x_train is a subset of train data , with val data taken out
## we can rejoin it again but well too uch back work 
model.fit(x_train, y_train)


[20:10:20] INFO: /workspace/src/learner.cc:215: Tree method is selected to be 'hist', which uses a single updater grow_quantile_histmaker.
[20:10:20] INFO: /workspace/src/tree/updater_quantile_hist.cc:63: Generating gmat: 0.0106621 sec
[20:10:20] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 62 extra nodes, 0 pruned nodes, max_depth=5
[20:10:20] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 0 pruned nodes, max_depth=5
[20:10:20] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned nodes, max_depth=5
[20:10:20] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=5
[20:10:20] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=5
[20:10:20] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 56 extra nodes, 0 pruned nodes, ma

XGBRegressor(colsample_bytree=0.4, learning_rate=0.04, max_depth=5,
             min_child_weight=3, n_estimators=500, objective='reg:squarederror',
             reg_lambda=0.02, subsample=0.4, tree_method='hist', verbosity=2)

## Submit Final Predictions 

In [26]:
y_pred = model.predict(x_test)
df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv') 
output = pd.DataFrame({'Id': df.Id,
                       'SalePrice': y_pred})
output.to_csv('submission.csv', index=False)

In [27]:
output.head()

,Id,SalePrice
0,1461,133467.171875
1,1462,160567.953125
2,1463,184239.390625
3,1464,192461.203125
4,1465,184111.281250
